## Arizona State University-Tempe

This script serves as a basic tutorial for extracting courses of interest from a university. This is by no means the only (or even best way) to go about this process—so if you come up with a process that works better, feel free to implement! If you're unfamiliar with any of the libraries, the comments below annotate reasoning behind each.

In [69]:
import sys
import pandas as pd
import numpy as np
import time
import re
import urllib.request #handles urls
from urllib.request import urlopen
import urllib.parse 
import linkGrabber #extracts urls
import json #encodes/decodes json 
import csv 
import requests #downloads a webpage to scrape
from bs4 import BeautifulSoup, NavigableString, Tag #beautifulsoup pulls data from HTML
import nltk #NLP tasks
from nltk import word_tokenize
from nltk.stem import PorterStemmer #removes word endings
stemmer = PorterStemmer()

The first thing we want to do is set up a function for standard preprocessing. It's also useful to list all of the URLs we'll need to send requests to before scraping. We want all courses within a 2 year *academic* calendar (as opposed to an annual calendar). 

In [70]:
#keyword preprocessing
def preprocess(keyword):
    keyword = keyword.lower() #lowercase
    keyword = word_tokenize(keyword) #tokenize
    for word in keyword:
        keyword = stemmer.stem(word) #stem 
    return (keyword)

Next, we'll want to import our keyword csv, split our keyword lists, and preprocess them. The way the csv is set up, we'll want to split the words that are indicated as technical (`T`) or normative (`N`) and that we've chosen to include (`Y`). You'll notice that preprocessing is useful for some of our words but not for others. Here, we've chosen to manually alter words that are not usefully preprocessed. In this case, it means replacing instances of words that are stemmed to end in i.

[regex is a bitch here]

In [71]:
#import keywords
keywords = pd.read_csv("../keywords.csv")
technical = keywords[(keywords['Technical/Normative']=='T') & (keywords['Include']=='Y')].Keyword
normative = keywords[(keywords['Technical/Normative']=='N') & (keywords['Include']=='Y')].Keyword
normative = [preprocess(i) for i in normative]
technical = [preprocess(i) for i in technical] 

#replace keywords of interest
normative = [w.replace('privaci', 'privac') for w in normative]
normative = [w.replace('democraci', 'democra') for w in normative]
normative = [w.replace('equiti', 'equit') for w in normative]
normative = [w.replace('histori', 'histor') for w in normative]
normative = [w.replace('justice', 'justic') for w in normative]
normative = [w.replace('liberti', 'libert') for w in normative]
normative = [w.replace('philosophi', 'philosoph') for w in normative]
normative = [w.replace('societi', 'societ') for w in normative]
normative = [w.replace('polici', 'polic') for w in normative]

technical = [w.replace('ai', '^ai') for w in technical]
technical = [w.replace('cs', '^cs') for w in technical]
technical = [w.replace('ict', '^ict') for w in technical]
technical = [w.replace('ml', '^ml') for w in technical]
technical = [w.replace('nlp', '^nlp') for w in technical]

print(normative)
print(technical)

['account', 'critic', 'democra', 'discrimin', 'equal', 'equit', 'ethic', 'fair', 'femin', 'gender', 'govern', 'histor', 'inequ', 'justic', 'law', 'legal', 'libert', 'moral', 'norm', 'philosoph', 'polit', 'power', 'privac', 'race', 'religi', 'respons', 'right', 'secur', 'social', 'societ', 'surveil', 'transpar', 'valu', 'polic']
['^ai', 'algorithm', 'analyt', 'intellig', 'automat', 'code', 'comput', '^cs', 'cyber', 'data', 'digit', '^ict', 'inform', 'intelligen', 'internet', 'machin', '^ml', 'process', '^nlp', 'platform', 'program', 'robot', 'softwar', 'system', 'technolog']


The process behind extracting relevant courses works in two steps:
1. First, we want to find and extract all courses that contain any instance of a normative keyword.
2. Then, we want search within these courses to see if it also contains a technical keyword.

We initialize a data frame with columns for all of the course items we want to extract. It probably makes the most sense to standardize these feature names across all university scripts so that they're easier to merge in the final compiled dataset for all universities. Our items of interest are:
* The course title: `title`
* The department and course number: `dept_num`
* The course description: `description`
* The number of credits for the course: `credits`
* The course instructor: `instructor`
* The link to the course syllabus (if applicable): `syllabus`
* The university the course is extracted from: `university`
* The term that the course is offered during (fall, spring, summer / year): `term`
* The keyword that triggered the extraction (this is for auditing purposes): `keyword`

In [72]:
asu_list = []

from selenium import webdriver
from selenium.webdriver.support.ui import Select
driver = webdriver.Chrome()

#course catalog URLs - 2 academic years 

terms = ['Fall 2017',
         'Spring 2018', 
         'Summer 2018', 
         'Fall 2018', 
         'Spring 2019', 
         'Summer 2019']

# previousTerms = ['Fall 2017']

for term in terms:
    print('--------------------')
    print('--------------------')
    print(term)
    
    for word in normative:
        #previous term
        if term == 'Fall 2017':
            driver.get("https://webapp4.asu.edu/catalog/?tlst=archives")
        else:
            driver.get("https://webapp4.asu.edu/catalog/classlist")
        time.sleep(2)
        
        select = Select(driver.find_element_by_xpath('//*[@id="term"]'))
        select.select_by_visible_text(term)

#         # for word in normative:
#         subject_field = driver.find_element_by_xpath('//*[@id="subj_id"]') 
#         subject_select = Select(subject_field)

#         #select all subjects
#         for subject in subject_field.find_elements_by_tag_name('option'):
#             subject_select.select_by_visible_text(subject.text)

        text_input = driver.find_element_by_xpath('//*[@id="keyword"]')
        
        print('--------------------')
        print(word)
        
        text_input.send_keys(word)
        
        #select tempe campus
        select_campus = driver.find_element_by_xpath('//*[@id="location-button"]').click()
        select_tempe = driver.find_element_by_xpath('//*[@id="TEMPE"]').click()
        
        #select all courses
        driver.find_element_by_xpath('//*[@id="searchTypeAllClass"]').click()
        
        time.sleep(2)
        
        get_course = driver.find_element_by_xpath('//*[@id="go_and_search"]')
        get_course.click()
        time.sleep(5)
        
#         all_courses = driver.find_elements_by_xpath('//*[@id="CatalogList"]/tbody')
        
        dept_nums = driver.find_elements_by_class_name('subjectNumberColumnValue')
        titles = driver.find_elements_by_class_name('titleColumnValue')
        instructors = driver.find_elements_by_class_name('instructorListColumnValue')
        all_credits = driver.find_elements_by_class_name('hoursColumnValue')

        for dept_num, title, instructor, credits in zip(dept_nums, titles, instructors, all_credits):
            asu_dict = {}
            asu_dict['dept_num'] = dept_num.text
            asu_dict['credits'] = credits.text
            asu_dict['instructor'] = instructor.text
            asu_dict['title'] = title.text
#             asu_dict['description'] = description.text
            asu_dict['term'] = term
            asu_dict['keyword'] = word
            asu_dict['university'] = 'arizona state university-tempe'
            
#           keyword search, so make sure words are also present in titles
            if word.lower() in title.text.lower():
                asu_list.append(asu_dict)
            
driver.close()

--------------------
--------------------
Spring 2018
--------------------
account
--------------------
critic
--------------------
democra
--------------------
discrimin
--------------------
equal
--------------------
equit
--------------------
ethic
--------------------
fair
--------------------
femin
--------------------
gender
--------------------
govern
--------------------
histor
--------------------
inequ
--------------------
justic
--------------------
law
--------------------
legal
--------------------
libert
--------------------
moral
--------------------
norm
--------------------
philosoph
--------------------
polit
--------------------
power
--------------------
privac
--------------------
race
--------------------
religi
--------------------
respons
--------------------
right
--------------------
secur
--------------------
social
--------------------
societ
--------------------
surveil
--------------------
transpar
--------------------
valu
--------------------
polic
-----

In [74]:
asu = pd.DataFrame(asu_list)
asu

,credits,dept_num,instructor,keyword,term,title,university
0,3,ACC 231,Wolfe,account,Spring 2018,Uses of Accounting Info I,arizona state university-tempe
1,3,ACC 231,Wolfe,account,Spring 2018,Uses of Accounting Info I,arizona state university-tempe
2,3,ACC 232,Srinivasan,account,Spring 2018,Financial Accounting I,arizona state university-tempe
3,3,ACC 232,Shields,account,Spring 2018,Financial Accounting I,arizona state university-tempe
4,3,ACC 232,Shields,account,Spring 2018,Financial Accounting I,arizona state university-tempe
5,3,ACC 241,Geiger,account,Spring 2018,Uses of Accounting Info II,arizona state university-tempe
6,3,ACC 261,Fox,account,Spring 2018,Honors Fund of Fin Accounting,arizona state university-tempe
7,3,ACC 261,Fox,account,Spring 2018,Honors Fund of Fin Accounting,arizona state university-tempe
8,3,ACC 271,Anderson,account,Spring 2018,Honors Fund of Mgr Accounting,arizona state university-tempe
9,3,ACC 360,Samuelson,account,Spring 2018,Ethics for Prof Accountants,arizona state university-tempe


The loop below executes part 1 of our extraction. It's long and kind of messy (sorry), so feel free to play around with the structure if you'd like. The key tasks here are to extract our items of interest based on our search queries and append them to our data frame.

Now that we've extracted all courses containing a normative keyword of interest, we need to filter our courses to only return titles that contain a normative AND a technical keyword. This is the case for all words except instances of our preprocessed `privac` and `secur`, for which we want to return all courses, even if they don't contain two keywords. To do this, we'll split the courses into two data frames, apply our respective conditions, and then merge them back together. 

In [75]:
exceptions = asu.loc[(asu['keyword']=='privac') | (asu['keyword'] =='secur')]
exceptions

,credits,dept_num,instructor,keyword,term,title,university
311,3,CIS 540,Steinbart,secur,Spring 2018,Information Security & Control,arizona state university-tempe
312,3,CSE 543,Yau,secur,Spring 2018,Info Assurance & Security,arizona state university-tempe
313,3,CSE 545,Doupe,secur,Spring 2018,Software Security,arizona state university-tempe
314,3,CSE 548,Huang,secur,Spring 2018,Adv Computer Network Security,arizona state university-tempe
315,3,FIN 421,Schreindorfer,secur,Spring 2018,Security Analys Portfolio Mgmt,arizona state university-tempe
316,3,FIN 421,Schreindorfer,secur,Spring 2018,Security Analys Portfolio Mgmt,arizona state university-tempe
317,3,FIN 421,Bhattacharya,secur,Spring 2018,Security Analys Portfolio Mgmt,arizona state university-tempe
318,3,FIN 421,Schreindorfer,secur,Spring 2018,Security Analys Portfolio Mgmt,arizona state university-tempe
319,3,FIN 421,Smith,secur,Spring 2018,Security Analys Portfolio Mgmt,arizona state university-tempe
320,3,FIN 421,Bonadurer,secur,Spring 2018,Security Analys Portfolio Mgmt,arizona state university-tempe


In [76]:
#loop through technical keyword list, extract relevant titles
for word in technical:
    df = asu[asu['title'].str.contains(word, flags = re.IGNORECASE)]
    df['keyword2'] = word
    
#join keyword cols
df["keyword"] = df["keyword"].map(str) + "," + df["keyword2"]
df = df.drop(columns="keyword2")

df

/Users/Shiying/.pyenv/versions/3.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/Shiying/.pyenv/versions/3.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,credits,dept_num,instructor,keyword,term,title,university
348,3,MCO 494,Kwon,"social,technolog",Spring 2018,"Topic: Technology, Digital Media and Social Is...",arizona state university-tempe
349,3,MCO 598,Kwon,"social,technolog",Spring 2018,"Topic: Technology, Digital Media and Social Is...",arizona state university-tempe
371,1,HSD 591,Fisher,"societ,technolog",Spring 2018,"Topic: Science, Technology and Society Reading...",arizona state university-tempe
430,3,HSD 502,Maynard,"polic,technolog",Spring 2018,Adv Science &Technology Policy\n(SEM),arizona state university-tempe
437,3,POS 572,Maynard,"polic,technolog",Spring 2018,Adv Science &Technology Policy\n(SEM),arizona state university-tempe
604,3,FIS 333,Frow,"govern,technolog",Fall 2018,Governing Emerging Technologie,arizona state university-tempe
674,3,LAW 394,Juetten,"law,technolog",Fall 2018,Topic: Technology Law,arizona state university-tempe
940,3,FIS 336,Farooque,"polic,technolog",Fall 2018,Science and Technology Policy,arizona state university-tempe
947,3,HSD 501,Cook-Deegan,"polic,technolog",Fall 2018,Science & Technology Policy\n(SEM),arizona state university-tempe
954,3,POS 571,Cook-Deegan,"polic,technolog",Fall 2018,Science & Technology Policy\n(SEM),arizona state university-tempe


NOTE: the above cell is likely not the best nor most simple way to execute this step! Feel free to take special liberties here. It's probably wise to pick out a few titles that you know should be returned manually, then check to see if the script is working as desired. 

In [77]:
#combine dfs 
asu = pd.concat([df, exceptions])
asu

,credits,dept_num,instructor,keyword,term,title,university
348,3,MCO 494,Kwon,"social,technolog",Spring 2018,"Topic: Technology, Digital Media and Social Is...",arizona state university-tempe
349,3,MCO 598,Kwon,"social,technolog",Spring 2018,"Topic: Technology, Digital Media and Social Is...",arizona state university-tempe
371,1,HSD 591,Fisher,"societ,technolog",Spring 2018,"Topic: Science, Technology and Society Reading...",arizona state university-tempe
430,3,HSD 502,Maynard,"polic,technolog",Spring 2018,Adv Science &Technology Policy\n(SEM),arizona state university-tempe
437,3,POS 572,Maynard,"polic,technolog",Spring 2018,Adv Science &Technology Policy\n(SEM),arizona state university-tempe
604,3,FIS 333,Frow,"govern,technolog",Fall 2018,Governing Emerging Technologie,arizona state university-tempe
674,3,LAW 394,Juetten,"law,technolog",Fall 2018,Topic: Technology Law,arizona state university-tempe
940,3,FIS 336,Farooque,"polic,technolog",Fall 2018,Science and Technology Policy,arizona state university-tempe
947,3,HSD 501,Cook-Deegan,"polic,technolog",Fall 2018,Science & Technology Policy\n(SEM),arizona state university-tempe
954,3,POS 571,Cook-Deegan,"polic,technolog",Fall 2018,Science & Technology Policy\n(SEM),arizona state university-tempe


In [ ]:
asu = asu[['title', 'dept_num', 'credits', 'instructor', 'university', 'term', 'keyword']]
asu

Lastly, we want to export our csv. Ideally, all csv files should be written to the courses directory in our repository. 

In [ ]:
#export as csv
asu.to_csv('../courses/30-Arizona State University-Tempe.csv', index=False)